In [18]:
import os
import json
import tqdm
import requests

from bs4 import BeautifulSoup

In [2]:
URL = 'https://www.bbc.com/portuguese/search?q=coronav%C3%ADrus'

In [3]:
def retrieve_urls(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    next_page = soup.find('a', class_='ws-search-pagination__link next')
    urls = []
    
    while next_page is not None:
        urls.append([a['href'] for a in soup.find_all('a', class_='hard-news-unit__headline-link')])
        next_url = url + '&' + next_page['href'].split('&')[1]
        page = requests.get(next_url)
        soup = BeautifulSoup(page.content, 'html.parser')
        next_page = soup.find('a', class_='ws-search-pagination__link next')
    
    return [url for url_list in urls for url in url_list]

In [4]:
urls = retrieve_urls(URL)

In [19]:
files_urls = []
for i, url in enumerate(urls):
    files_urls.append({'filename': 'url{}.txt'.format(i), 'url': url})

In [20]:
with open('data/urls.json', 'w') as json_file:
    json_file.write(json.dumps(files_urls, indent=2))

In [21]:
for file_url in tqdm.tqdm(files_urls):
    with open(os.path.join('data', file_url['filename']), 'w') as txt_file:
        txt_file.write(requests.get(file_url['url']).text)

100%|██████████| 350/350 [10:53<00:00,  1.87s/it]
